## Import necessary libraries

In [ ]:
import pandas as pd
import joblib
from google.colab import files
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor




# Load the dataset



In [ ]:
uploaded = files.upload()


In [ ]:
df = pd.read_csv('yield_df.csv')

# Initial Exploration of the dataset


In [ ]:
# display the shape of the DataFrame
print("Shape of the DataFrame:", df.shape)

# display first 5 rows of the dataframe to verify it has been loaded correctly
display(df.head())

# Display information about the DataFrame (like data types, non-null values, and etc.)
print("\nDataFrame Info:")
df.info()

# Display descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
display(df.describe())

# Display descriptive statistics for categorical columns
print("\nDescriptive Statistics for Categorical Columns:")
display(df.describe(include='object'))

## Preprocess Dataset to get it ready for deeper exploratory analysis


In [ ]:
# Drop the 'Unnamed: 0' column as it is likely an index
df = df.drop('Unnamed: 0', axis=1)

# Display the first few rows to confirm the column is dropped
display(df.head())

In [ ]:
# Check for missing values in each column
print("Missing values per column:")
print(df.isnull().sum())

In [ ]:
# identify for duplicate rows
duplicate_rows = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_rows}")

# drop duplicate rows
df = df.drop_duplicates()
print(f"Number of rows after removing duplicates: {df.shape[0]}")

## Prepare and Save Original DataFrame for Streamlit and SQL

To allow the Streamlit app to display original historical data, we need to save the DataFrame in its state *before* one-hot encoding. This ensures the app can access the non-encoded `Area` and `Item` names along with the numerical features and actual yield.

In [ ]:
# Create a copy of the DataFrame before one-hot encoding to preserve original categorical values
# This `df` state is after dropping 'Unnamed: 0' and duplicates, but before one-hot encoding.
original_df_for_streamlit = df.copy()

# Save this DataFrame
joblib.dump(original_df_for_streamlit, 'original_df_for_streamlit.pkl')

print("Original DataFrame (pre-one-hot encoding) saved as 'original_df_for_streamlit.pkl'!")

## Load Cleaned DataFrame into SQLite

### Subtask:
Establish an in-memory SQLite database connection and load the currently processed and cleaned `df` (crop yield data) into a SQL table.


In [ ]:
import sqlite3

# Create an in-memory SQLite database connection
conn = sqlite3.connect(':memory:')

# Save the df DataFrame to a new SQL table named 'crop_yield_data'
original_df_for_streamlit.to_sql('crop_yield_data', conn, if_exists='replace', index=False)

print("DataFrame loaded into SQLite table 'crop_yield_data'.")

# Display the schema of the 'crop_yield_data' table
print("\nSchema of 'crop_yield_data' table:")
display(pd.read_sql_query("PRAGMA table_info(crop_yield_data);", conn))

# Task
Perform SQL data integrity checks on the `crop_yield_data` table by counting null values for each column and unique values for the 'Area', 'Item', and 'Year' columns. Display the results in separate pandas DataFrames.

## Perform SQL Schema Inspection and Integrity Checks

### Subtask:
Perform SQL data integrity checks on the `crop_yield_data` table by counting null values for each column and unique values for the 'Area', 'Item', and 'Year' columns.


In [ ]:
null_counts_query = '''
SELECT
    SUM(CASE WHEN Area IS NULL THEN 1 ELSE 0 END) AS Area_Nulls,
    SUM(CASE WHEN Item IS NULL THEN 1 ELSE 0 END) AS Item_Nulls,
    SUM(CASE WHEN Year IS NULL THEN 1 ELSE 0 END) AS Year_Nulls,
    SUM(CASE WHEN "hg/ha_yield" IS NULL THEN 1 ELSE 0 END) AS hg_ha_yield_Nulls,
    SUM(CASE WHEN average_rain_fall_mm_per_year IS NULL THEN 1 ELSE 0 END) AS average_rain_fall_mm_per_year_Nulls,
    SUM(CASE WHEN pesticides_tonnes IS NULL THEN 1 ELSE 0 END) AS pesticides_tonnes_Nulls,
    SUM(CASE WHEN avg_temp IS NULL THEN 1 ELSE 0 END) AS avg_temp_Nulls
FROM
    crop_yield_data;
'''

null_counts_df = pd.read_sql_query(null_counts_query, conn)
print("Null counts for each column:")
display(null_counts_df)

unique_counts_query = '''
SELECT
    COUNT(DISTINCT Area) AS Unique_Areas,
    COUNT(DISTINCT Item) AS Unique_Items,
    COUNT(DISTINCT Year) AS Unique_Years
FROM crop_yield_data;
'''

unique_counts_df = pd.read_sql_query(unique_counts_query, conn)
print("\nUnique counts for Area, Item, and Year:")
display(unique_counts_df)

## Execute Advanced SQL Analytical Queries

### Subtask:
Craft and execute advanced SQL queries on the `crop_yield_data` table to derive new insights, such as finding top areas for specific crop yields, analyzing year-over-year changes, and calculating average yields under certain environmental conditions.


In [ ]:
import pandas as pd

# 1. SQL query to find the top 5 areas with the highest average 'Maize' yield
top_maize_areas_query = '''
SELECT
    Area,
    AVG("hg/ha_yield") AS Average_Yield
FROM
    crop_yield_data
WHERE
    Item = 'Maize'
GROUP BY
    Area
ORDER BY
    Average_Yield DESC
LIMIT 5;
'''
top_maize_areas_df = pd.read_sql_query(top_maize_areas_query, conn)
print("Top 5 areas with highest average Maize yield:")
display(top_maize_areas_df)

# 2. SQL query to calculate the year-over-year yield change for 'Wheat' in 'India'
india_wheat_yoy_query = '''
WITH IndiaWheatYield AS (
    SELECT
        Year,
        "hg/ha_yield" AS Current_Yield,
        LAG("hg/ha_yield", 1, 0) OVER (ORDER BY Year) AS Previous_Year_Yield
    FROM
        crop_yield_data
    WHERE
        Area = 'India' AND Item = 'Wheat'
)
SELECT
    Year,
    Current_Yield,
    Previous_Year_Yield,
    (Current_Yield - Previous_Year_Yield) AS YoY_Change
FROM
    IndiaWheatYield
WHERE
    Previous_Year_Yield != 0 -- Exclude the first year where Previous_Year_Yield is 0
ORDER BY
    Year;
'''
india_wheat_yoy_df = pd.read_sql_query(india_wheat_yoy_query, conn)
print("\nYear-over-year yield change for Wheat in India:")
display(india_wheat_yoy_df)

# 3. SQL query to find the average yield for each crop ('Item') when avg_temp is above 25 degrees Celsius
high_temp_yield_query = '''
SELECT
    Item,
    AVG("hg/ha_yield") AS Average_Yield_High_Temp
FROM
    crop_yield_data
WHERE
    avg_temp > 25
GROUP BY
    Item
ORDER BY
    Average_Yield_High_Temp DESC;
'''
high_temp_yield_df = pd.read_sql_query(high_temp_yield_query, conn)
print("\nAverage yield for each crop when average temperature is above 25°C:")
display(high_temp_yield_df)

# Deeper exploration of the dataset
Explore the dataset further by visualizing the distribution of numerical and categorical features, examining the relationships between features and the target variable, and analyzing correlations between numerical features.

## Visualize the distribution of numerical features




In [ ]:
numerical_cols = ['Year', 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']

plt.figure(figsize=(15, 10))

for i, col in enumerate(numerical_cols):
    plt.subplot(2, 3, i + 1)
    sns.histplot(data=df, x=col, kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## Explore the relationship between numerical features and the target variable



In [ ]:
# create scatter plots for key environmental/management factors vs. yield
plt.figure(figsize=(18, 5))

# scatter plot: Average Rainfall vs. Yield
plt.subplot(1, 3, 1)
sns.scatterplot(x='average_rain_fall_mm_per_year', y='hg/ha_yield', data=df)
plt.title("Average Rainfall vs. Yield")
plt.xlabel("Average Rainfall (mm/year)")
plt.ylabel("Yield (hg/ha)")
plt.grid(True)

# scatter plot: Pesticides Tonnes vs. Yield
plt.subplot(1, 3, 2)
sns.scatterplot(x='pesticides_tonnes', y='hg/ha_yield', data=df)
plt.title("Pesticides Tonnes vs. Yield")
plt.xlabel("Pesticides Tonnes")
plt.ylabel("Yield (hg/ha)")
plt.grid(True)

# scatter plot: Average Temperature vs. Yield
plt.subplot(1, 3, 3)
sns.scatterplot(x='avg_temp', y='hg/ha_yield', data=df)
plt.title("Average Temperature vs. Yield")
plt.xlabel("Average Temperature (°C)")
plt.ylabel("Yield (hg/ha)")
plt.grid(True)

plt.tight_layout()
plt.show()

## Investigate the relationship between categorical features and the target variable




In [ ]:
# identify the top 10 most frequent areas based on the 'Area' column's value counts.
top_10_areas = df['Area'].value_counts().nlargest(10).index.tolist()

# Filtere DataFrame to include only the top 10 areas
df_top_areas = df[df['Area'].isin(top_10_areas)]

#  box plot to visualize the distribution of 'hg/ha_yield' for the top 10 area.
plt.figure(figsize=(14, 7))
sns.boxplot(x='Area', y='hg/ha_yield', data=df_top_areas, order=top_10_areas)
plt.title('Distribution of Yield (hg/ha) Across Top 10 Areas')
plt.xlabel('Area')
plt.ylabel('Yield (hg/ha)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

#  box plot to visualize the distribution of 'hg/ha_yield' for all unique 'Item' categories.
plt.figure(figsize=(14, 7))
sns.boxplot(x='Item', y='hg/ha_yield', data=df, order=df['Item'].value_counts().index)
plt.title('Distribution of Yield (hg/ha) Across All Crop Items')
plt.xlabel('Item')
plt.ylabel('Yield (hg/ha)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

## Explore correlations between numerical features



In [ ]:
# select numerical columns
numerical_cols = ['Year', 'hg/ha_yield', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']
df_numerical = df[numerical_cols]

# clculate the correlation matrix
correlation_matrix = df_numerical.corr()

# make the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Numerical Features')
plt.show()

# Encoding categorical variables


In [ ]:
# Perform one-hot encoding for 'Area' and 'Item' columns
df = pd.get_dummies(df, columns=['Area', 'Item'], drop_first=False)

# Display the first few rows of the modified DataFrame to see the new columns
display(df.head())

# Display the shape of the DataFrame to see the increase in columns
print("\nShape of the DataFrame after one-hot encoding:", df.shape)

### Split data



In [ ]:
X = df.drop('hg/ha_yield', axis=1)
y = df['hg/ha_yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



## Model selection and evaluation

### Subtask:
Train and evaluate multiple models to compare their performance.


In [ ]:
X = df.drop('hg/ha_yield', axis=1)
y = df['hg/ha_yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# make the columnsd aligned because crucial for consistent feature sets between train and test
train_cols = X_train.columns
test_cols = X_test.columns

missing_in_test = set(train_cols) - set(test_cols)
for c in missing_in_test:
    X_test[c] = 0

missing_in_train = set(test_cols) - set(train_cols)
for c in missing_in_train:
    X_train[c] = 0

X_test = X_test[train_cols] #  the order of columns is the same

# intiialize and train the models again
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(max_iter=10000), # Increased max_iter
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42)
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse) # Calculate RMSE
    mae = mean_absolute_error(y_test, y_pred) # Calculate MAE
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "RMSE": rmse, "MAE": mae, "R-squared": r2} # storage of all metrics
    print(f"{name} - MSE: {mse:.2f}, RMSE: {rmse:.2f}, MAE: {mae:.2f}, R-squared: {r2:.2f}\n")

print("\nModel Evaluation Results:")
for name, metrics in results.items():
    print(f"{name}:")
    print(f"  MSE: {metrics['MSE']:.2f}")
    print(f"  RMSE: {metrics['RMSE']:.2f}")
    print(f"  MAE: {metrics['MAE']:.2f}")
    print(f"  R-squared: {metrics['R-squared']:.2f}")

## Justify model choice

### stask:
Based on the evaluation, select the most appropriate model and justify the choice.


In [ ]:
# convedrt results dictionary to a pandas DataFrame for easier plotting
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df = results_df.reset_index().rename(columns={'index': 'Model'})

# make the DataFrame to have a single column for metrics (MSE, RMSE, MAE, and R-squared)
melted_results_df = results_df.melt(id_vars='Model', var_name='Metric', value_name='Value')

# create bar plots for MSE, RMSE, MAE and R-squared
plt.figure(figsize=(18, 8))

plt.subplot(2, 2, 1)
sns.barplot(x='Model', y='Value', data=melted_results_df[melted_results_df['Metric'] == 'MSE'])
plt.title('Model Performance Comparison (MSE)')
plt.ylabel('Mean Squared Error (MSE)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.subplot(2, 2, 2)
sns.barplot(x='Model', y='Value', data=melted_results_df[melted_results_df['Metric'] == 'RMSE'])
plt.title('Model Performance Comparison (RMSE)')
plt.ylabel('Root Mean Squared Error (RMSE)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.subplot(2, 2, 3)
sns.barplot(x='Model', y='Value', data=melted_results_df[melted_results_df['Metric'] == 'MAE'])
plt.title('Model Performance Comparison (MAE)')
plt.ylabel('Mean Absolute Error (MAE)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

plt.subplot(2, 2, 4)
sns.barplot(x='Model', y='Value', data=melted_results_df[melted_results_df['Metric'] == 'R-squared'])
plt.title('Model Performance Comparison (R-squared)')
plt.ylabel('R-squared')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()


plt.show()

## K-fold cross validation

In [ ]:
# instantiate KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

cv_results = {}

for name, model in models.items():
    print(f"Performing cross-validation for {name}...")

    # cross-validation for MSE (negative)
    mse_scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    mean_mse = -mse_scores.mean() # Convert to positive MSE
    std_mse = mse_scores.std()

    # cross-validation for R-squared
    r2_scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
    mean_r2 = r2_scores.mean()
    std_r2 = r2_scores.std()

    cv_results[name] = {
        "Mean MSE": mean_mse,
        "Std MSE": std_mse,
        "Mean R-squared": mean_r2,
        "Std R-squared": std_r2
    }

    print(f"{name} - Mean MSE: {mean_mse:.2f} (± {std_mse:.2f}), Mean R-squared: {mean_r2:.2f} (± {std_r2:.2f})\n")

print("\nCross-Validation Results:")
for name, metrics in cv_results.items():
    print(f"{name}:")
    print(f"  Mean MSE: {metrics['Mean MSE']:.2f} (± {metrics['Std MSE']:.2f})")
    print(f"  Mean R-squared: {metrics['Mean R-squared']:.2f} (± {metrics['Std R-squared']:.2f})")

## Hyperparameter Tuning for Random Forest Regressor using GridSearchCV


Optimize the hyperparameters of the Random Forest Regressor using GridSearchCV.

In [ ]:
# define the parameter grid for GridSearchCV
param_grid = {
    'n_estimators': [100, 200],  # Reduced number of trees
    'max_depth': [10, 20],      # Reduced options for max depth
    'min_samples_split': [2, 5],  # Reduced options for min samples split
    'min_samples_leaf': [1, 2]     # Reduced options for min samples leaf
}

# initialize GridSearchCV
# using the Random Forest Regressor
grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=3,       # reduced cross-validation folds for faster execution
                           scoring='neg_mean_squared_error', # score using negative MSE
                           n_jobs=-1)   # Use all available CPU cores

print("Performing GridSearchCV for Random Forest Regressor...")

#  the grid search on the training data
grid_search.fit(X_train, y_train)

print("GridSearchCV complete.")

#  the best parameters found
print("\nBest parameters found: ", grid_search.best_params_)

#  the best cross-validation score (negative MSE)
print("\nBest cross-validation score (negative MSE): ", grid_search.best_score_)

#  the best estimator (the trained model with the best parameters)
best_rf_model_tuned = grid_search.best_estimator_

# evaluate the best tuned model on the test set
y_pred_best_rf_tuned = best_rf_model_tuned.predict(X_test)
mse_best_rf_tuned = mean_squared_error(y_test, y_pred_best_rf_tuned)
rmse_best_rf_tuned = np.sqrt(mse_best_rf_tuned)
mae_best_rf_tuned = mean_absolute_error(y_test, y_pred_best_rf_tuned)
r2_best_rf_tuned = r2_score(y_test, y_pred_best_rf_tuned)

print("\nEvaluation of the best tuned Random Forest model on the test set:")
print(f"  MSE: {mse_best_rf_tuned:.2f}")
print(f"  RMSE: {rmse_best_rf_tuned:.2f}")
print(f"  MAE: {mae_best_rf_tuned:.2f}")
print(f"  R-squared: {r2_best_rf_tuned:.2f}")

In [ ]:

best_model = best_rf_model_tuned


area_cols = [col for col in X_train.columns if col.startswith('Area_')]
item_cols = [col for col in X_train.columns if col.startswith('Item_')]


area_results = {}
item_results = {}


print("Best tuned model selected for potential further analysis:", type(best_model).__name__)



## Best Model Performance Metrics

## Visualization of Untuned vs. Tuned Random Forest Regressor Performance



In [ ]:
#  the performance metrics for the untuned Random Forest Regressor

untuned_rf_metrics = results["Random Forest Regressor"]

# the performance metrics for the tuned Random Forest Regressor

tuned_rf_metrics = {
    "MSE": mse_best_rf_tuned,
    "RMSE": rmse_best_rf_tuned,
    "MAE": mae_best_rf_tuned,
    "R-squared": r2_best_rf_tuned
}

# Prepare data for plotting
metrics_data = {
    "Metric": ["MSE", "RMSE", "MAE", "R-squared"] * 2,
    "Value": [
        untuned_rf_metrics["MSE"], untuned_rf_metrics["RMSE"], untuned_rf_metrics["MAE"], untuned_rf_metrics["R-squared"],
        tuned_rf_metrics["MSE"], tuned_rf_metrics["RMSE"], tuned_rf_metrics["MAE"], tuned_rf_metrics["R-squared"]
    ],
    "Model": ["Untuned RF"] * 4 + ["Tuned RF"] * 4
}

metrics_df = pd.DataFrame(metrics_data)

# Create bar plots for each metric
plt.figure(figsize=(14, 8))

# Plot MSE
plt.subplot(2, 2, 1)
sns.barplot(x="Model", y="Value", data=metrics_df[metrics_df["Metric"] == "MSE"])
plt.title("MSE Comparison")
plt.ylabel("MSE")

# Plot RMSE
plt.subplot(2, 2, 2)
sns.barplot(x="Model", y="Value", data=metrics_df[metrics_df["Metric"] == "RMSE"])
plt.title("RMSE Comparison")
plt.ylabel("RMSE")

# Plot MAE
plt.subplot(2, 2, 3)
sns.barplot(x="Model", y="Value", data=metrics_df[metrics_df["Metric"] == "MAE"])
plt.title("MAE Comparison")
plt.ylabel("MAE")

# Plot R-squared
plt.subplot(2, 2, 4)
sns.barplot(x="Model", y="Value", data=metrics_df[metrics_df["Metric"] == "R-squared"])
plt.title("R-squared Comparison")
plt.ylabel("R-squared")

plt.tight_layout()
plt.show()

## Actual vs Predicted Crop Yield

In [ ]:
# get the best performing model (Random Forest Regressor)
# use the untuned Random Forest model for comparison
best_model = models["Random Forest Regressor"]

# make predictions on the test set
y_pred_best_model = best_model.predict(X_test)

# scatter plot of Actual vs. Predicted Yield
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred_best_model)
plt.xlabel("Actual Yield (hg/ha)")
plt.ylabel("Predicted Yield (hg/ha)")
plt.title("Actual vs. Predicted Crop Yield (Untuned Random Forest Regressor)")
plt.grid(True)
plt.show()

In [ ]:
# Load the original_df_for_streamlit for per-crop mean calculation
# This DataFrame retains the original 'Item' column before one-hot encoding.
original_df_for_streamlit = joblib.load('original_df_for_streamlit.pkl')
print("original_df_for_streamlit loaded for per-crop mean calculation.")

# Calculate the mean yield for each crop ('Item') using the original_df_for_streamlit
per_crop_mean_yield = original_df_for_streamlit.groupby('Item')['hg/ha_yield'].mean().to_dict()

# Save the dictionary of per-crop mean yields
joblib.dump(per_crop_mean_yield, 'per_crop_mean_yield.pkl')

print("Per-crop mean yield calculated and saved as 'per_crop_mean_yield.pkl'!")

## Mean Historical Yield per Crop Visualization

This graph displays the average historical yield for each crop type present in the dataset, providing a quick comparison of productivity across different items.

In [ ]:
# Load the per_crop_mean_yield dictionary
# Ensure 'per_crop_mean_yield.pkl' has been created and saved previously
per_crop_mean_yield = joblib.load('per_crop_mean_yield.pkl')


# Convert the dictionary to a DataFrame for easier plotting
yield_df = pd.DataFrame(list(per_crop_mean_yield.items()), columns=['Crop', 'Mean Yield (hg/ha)'])

# Sort for better visualization
yield_df = yield_df.sort_values(by='Mean Yield (hg/ha)', ascending=False)

# Create the bar plot
plt.figure(figsize=(12, 7))
sns.barplot(x='Mean Yield (hg/ha)', y='Crop', data=yield_df, palette='viridis', hue='Crop', legend=False)
plt.title('Mean Historical Yield per Crop')
plt.xlabel('Mean Yield (hg/ha)')
plt.ylabel('Crop Item')
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

## Residual plot

In [ ]:
# get the best performing model (Random Forest Regressor)
# use the untuned Random Forest model for comparison
best_model = models["Random Forest Regressor"]

# make predictions on the test set
y_pred_best_model = best_model.predict(X_test)

# calculate residuals
residuals = y_test - y_pred_best_model

# residual Plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_pred_best_model, y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel("Predicted Yield (hg/ha)")
plt.ylabel("Residuals")
plt.title("Residual Plot (Untuned Random Forest Regressor)")
plt.grid(True)
plt.show()

## Feature Importance

In [ ]:
# Get the best performing model (Random Forest Regressor)
# use the untuned Random Forest model for comparison
best_model = models["Random Forest Regressor"]

# get feature importances
# feature importances should be from the untuned model
feature_importances = best_model.feature_importances_

# create a DataFrame for better visualization
features_df = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})

# Sort by importance
features_df = features_df.sort_values(by='Importance', ascending=False)

# Display the top 10 most important features
display(features_df.head(10))

# Plot feature importances (top 10)
plt.figure(figsize=(12, 7))
sns.barplot(x='Importance', y='Feature', data=features_df.head(10))
plt.title("Top 10 Most Important Features (Untuned Random Forest Regressor)")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

## Save best trained model to prep Streamlit deployment


In [ ]:
# Save the best trained model
joblib.dump(best_rf_model_tuned, 'best_rf_model_tuned.pkl')

# Also save the list of columns from the training data
# This is crucial for ensuring the Streamlit app receives inputs in the correct format
joblib.dump(X_train.columns, 'training_columns.pkl')

# Save feature importances for visualization in Streamlit app
joblib.dump(features_df, 'feature_importances.pkl')

# Save mean and standard deviation of y_train for contextual plots
joblib.dump(y_train.mean(), 'y_train_mean.pkl')
joblib.dump(y_train.std(), 'y_train_std.pkl')

print("Tuned Random Forest model, training columns, feature importances, and y_train stats saved successfully!")

## requirements file for Streamlit App


In [ ]:
%%writefile requirements.txt
pandas
scikit-learn
streamlit
matplotlib
seaborn
numpy
altair==4.2.2


## Code for Streamlit App


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

st.set_page_config(layout='wide')

st.title('Crop Yield Prediction App')
st.write('Predict crop yield based on environmental factors and crop types.')

# Load the trained model and training columns
@st.cache_resource
def load_data():
    model = joblib.load('best_rf_model_tuned.pkl')
    training_columns = joblib.load('training_columns.pkl')
    feature_importances_df = joblib.load('feature_importances.pkl')
    per_crop_mean_yield = joblib.load('per_crop_mean_yield.pkl')
    original_df = joblib.load('original_df_for_streamlit.pkl') # Load original df
    return model, training_columns, feature_importances_df, per_crop_mean_yield, original_df

model, training_columns, feature_importances_df, per_crop_mean_yield, original_df = load_data()

# Extract unique Area and Item names from training_columns for select boxes
all_areas = sorted([col.replace('Area_', '') for col in training_columns if col.startswith('Area_')])
all_items = sorted([col.replace('Item_', '') for col in training_columns if col.startswith('Item_')])

# Sidebar for user input
st.sidebar.header('Input Features')

# User inputs
selected_area = st.sidebar.selectbox('Area', all_areas)
selected_item = st.sidebar.selectbox('Item', all_items)

year = st.sidebar.slider('Year', min_value=1990, max_value=2013, value=2010) # Adjusted max_value to 2013
rain_fall = st.sidebar.number_input('Average Rainfall (mm/year)', min_value=0.0, max_value=20000.0, value=1500.0)
pesticides = st.sidebar.number_input('Pesticides (tonnes)', min_value=0.0, max_value=100000.0, value=5000.0)
temperature = st.sidebar.number_input('Average Temperature (°C)', min_value=-20.0, max_value=40.0, value=20.0)


if st.sidebar.button('Predict Yield'):
    # Create a DataFrame for the current input, matching the structure of X_train
    input_data = pd.DataFrame(0, index=[0], columns=training_columns)

    # Populate numerical features
    input_data['Year'] = year
    input_data['average_rain_fall_mm_per_year'] = rain_fall
    input_data['pesticides_tonnes'] = pesticides
    input_data['avg_temp'] = temperature

    # Populate one-hot encoded categorical features
    if f'Area_{selected_area}' in input_data.columns:
        input_data[f'Area_{selected_area}'] = 1
    if f'Item_{selected_item}' in input_data.columns:
        input_data[f'Item_{selected_item}'] = 1

    # Ensure the order of columns is the same as during training
    input_data = input_data[training_columns]

    # Make prediction
    prediction = model.predict(input_data)[0]

    st.subheader('Prediction Result')

    # Display prediction prominently
    col1, col2 = st.columns([1, 2])
    with col1:
        st.metric("Predicted Yield", f"{prediction:,.2f} hg/ha")
    with col2:
        # Get historical mean for the selected crop
        historical_yield_for_item = per_crop_mean_yield.get(selected_item, 0) # Use .get with default 0 if item not found

        # --- New: Find and display historical data for comparison ---
        historical_record = original_df[
            (original_df['Area'] == selected_area) &
            (original_df['Item'] == selected_item) &
            (original_df['Year'] == year)
        ]

        plot_labels = ['Predicted', f'Avg Historical ({selected_item})']
        plot_values = [prediction, historical_yield_for_item]

        if not historical_record.empty:
            actual_yield = historical_record['hg/ha_yield'].iloc[0]
            historical_rain = historical_record['average_rain_fall_mm_per_year'].iloc[0]
            historical_pesticides = historical_record['pesticides_tonnes'].iloc[0]
            historical_temp = historical_record['avg_temp'].iloc[0]

            st.markdown(f"**Historical Data for {selected_item} in {selected_area} ({year}):**")
            st.write(f"- **Actual Yield:** {actual_yield:,.2f} hg/ha")
            st.write(f"- **Rainfall:** {historical_rain:,.2f} mm/year")
            st.write(f"- **Pesticides:** {historical_pesticides:,.2f} tonnes")
            st.write(f"- **Temperature:** {historical_temp:,.2f} °C")
            st.markdown("--- ")

            plot_labels.insert(1, 'Actual')
            plot_values.insert(1, actual_yield)

        else:
            st.info(f"No exact historical record found for {selected_item} in {selected_area} for {year}. Displaying predicted vs. average historical.")

        # Contextual plot: Predicted vs. Actual vs. Average Historical Yield
        fig_pred_vs_avg, ax_pred_vs_avg = plt.subplots(figsize=(6, 3))
        bars = ax_pred_vs_avg.bar(plot_labels, plot_values, color=['skyblue', 'lightgreen', 'lightgray'][:len(plot_labels)])
        ax_pred_vs_avg.set_ylabel('Yield (hg/ha)')
        ax_pred_vs_avg.set_title(f'Yield Comparison for {selected_item}')
        for bar in bars:
            yval = bar.get_height()
            ax_pred_vs_avg.text(bar.get_x() + bar.get_width()/2, yval + 500, round(yval, 0), ha='center', va='bottom', fontsize=8)
        st.pyplot(fig_pred_vs_avg)

    st.markdown("--- ")

# General Model Insights (Feature Importance)
st.subheader('Model Insights: Feature Importance')
with st.expander("View Feature Importance", expanded=False):
    fig_feature_imp, ax_feature_imp = plt.subplots(figsize=(10, 6))
    sns.barplot(x='Importance', y='Feature', data=feature_importances_df.head(10), ax=ax_feature_imp)
    ax_feature_imp.set_title("Top 10 Most Important Features")
    ax_feature_imp.set_xlabel("Importance")
    ax_feature_imp.set_ylabel("Feature")
    st.pyplot(fig_feature_imp)